In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model, layers
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
# model parameter 계산
model = Sequential([tf.keras.Input([5,]),
                    layers.Dense(20,activation='relu'),
                    layers.Dense(30,activation='relu'),
                    layers.Dense(10)])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 20)                120       
                                                                 
 dense_1 (Dense)             (None, 30)                630       
                                                                 
 dense_2 (Dense)             (None, 10)                310       
                                                                 
Total params: 1,060
Trainable params: 1,060
Non-trainable params: 0
_________________________________________________________________


2022-07-29 15:25:52.345863: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-29 15:25:53.537097: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9649 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1
2022-07-29 15:25:53.544595: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10139 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [2]:
(x_trainval, y_trainval), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
print(f"x_trainval: {type(x_trainval)}, {x_trainval.shape}")
print(f"x_test: {type(x_test)}, {x_test.shape}")

x_train, x_valid, y_train, y_valid = train_test_split(x_trainval, y_trainval, test_size=1/6, shuffle=True,
                                                      stratify=y_trainval, random_state=34)
print(f"x_train: {type(x_train)}, {x_train.shape}")
print(f"x_valid: {type(x_valid)}, {x_valid.shape}")

x_trainval: <class 'numpy.ndarray'>, (60000, 28, 28)
x_test: <class 'numpy.ndarray'>, (10000, 28, 28)
x_train: <class 'numpy.ndarray'>, (50000, 28, 28)
x_valid: <class 'numpy.ndarray'>, (10000, 28, 28)


In [3]:
x_train = x_train.astype('float32') / 255
x_valid = x_valid.astype('float32') / 255
x_test = x_test.astype('float32') / 255

print(x_train.shape)

x_train = x_train.reshape(-1, 28, 28, 1)
x_valid = x_valid.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

print(x_train.shape)

(50000, 28, 28)
(50000, 28, 28, 1)


In [9]:
class CNN(Model):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = layers.Conv2D(32, kernel_size=3, strides=(1,1), padding='same', activation=tf.nn.relu, input_shape=(28,28,1))
        self.maxpool1 = layers.MaxPool2D((2,2), strides=(1,1))
        self.conv2 = layers.Conv2D(64, kernel_size=3, strides=(1,1), padding='valid', activation=tf.nn.relu)
        self.maxpool1 = layers.MaxPool2D((2,2), strides=2)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(128)
        self.out = layers.Dense(10)
    
    def call(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)        
        x = self.flatten(x)        
        x = self.fc1(x)
        x = self.out(x)        

In [10]:
cnn = CNN()
cnn.summary()

ValueError: This model has not yet been built. Build the model first by calling `build()` or by calling the model on a batch of data.

In [5]:
cnn = Sequential([
    # Feature extraction
    layers.Conv2D(32, 3, strides=1, padding='same', activation='relu', input_shape=(28,28,1)),
    layers.MaxPool2D(3, strides=2, padding='same'),
    layers.Conv2D(64, 3, strides=1, padding='same', activation='relu'),
    layers.MaxPool2D(3, strides=2, padding='same'),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 3136)              0         
                                                                 
 dense_3 (Dense)             (None, 128)              

In [5]:
cnn.compile(loss='sparse_categorical_crossentropy',
            optimizer=tf.keras.optimizers.SGD(learning_rate=0.1),
            metrics=['accuracy'])

In [6]:
result = cnn.fit(x_train, y_train,
                 epochs=5,
                 batch_size=128,
                 validation_data=(x_train, y_train))

Epoch 1/5


2022-07-28 13:50:10.423523: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8401


391/391 [==============================] - 6s 9ms/step - loss: 0.4558 - accuracy: 0.8544 - val_loss: 0.1317 - val_accuracy: 0.9578
Epoch 2/5
391/391 [==============================] - 3s 8ms/step - loss: 0.0873 - accuracy: 0.9724 - val_loss: 0.0748 - val_accuracy: 0.9760
Epoch 3/5
391/391 [==============================] - 3s 8ms/step - loss: 0.0617 - accuracy: 0.9803 - val_loss: 0.0647 - val_accuracy: 0.9792
Epoch 4/5
391/391 [==============================] - 3s 8ms/step - loss: 0.0466 - accuracy: 0.9858 - val_loss: 0.0503 - val_accuracy: 0.9847
Epoch 5/5
391/391 [==============================] - 3s 8ms/step - loss: 0.0398 - accuracy: 0.9877 - val_loss: 0.0347 - val_accuracy: 0.9892


In [7]:
test_result = cnn.evaluate(x_test, y_test)
print(f"test loss: {test_result[0]}")
print(f"test acc: {test_result[1]}")

313/313 [==============================] - 1s 3ms/step - loss: 0.0404 - accuracy: 0.9869
test loss: 0.04038459435105324
test acc: 0.9868999719619751
